In [5]:
import pandas as pd
import numpy as np
import nltk
!pip install matplotlib.pyplot
from wordcloud import WordCloud
#import matplotlib.pyplot as plt

  Could not find a version that satisfies the requirement matplotlib.pyplot (from versions: )
No matching distribution found for matplotlib.pyplot


In [6]:
500/(100*300)

0.016666666666666666

In [ ]:
dialogues = pd.read_csv("dialogues_bigbangtheory.csv")

In [ ]:
dialogues.head()

In [ ]:
sheldon_dialogues = dialogues[dialogues['character']=='Sheldon']

In [ ]:
sheldon_dialogues.head()

In [ ]:
sheldon_dialogues.shape[0]

In [ ]:
input_string2 = ' '.join(sheldon_dialogues['dialogue'])
wc = WordCloud(background_color = "Black").generate(input_string2)
plt.imshow(wc)

In [ ]:
docs = sheldon_dialogues['dialogue'].str.lower().str.replace('[^a-z ]', '')
docs.head()

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')
stopwords[:5]

In [ ]:
stemmer = nltk.stem.PorterStemmer()

In [ ]:
docs=sheldon_dialogues['dialogue'].str.lower().str.replace('[^a-z ]','')
docs_clean2=[]
def clean_sentence(text):
    words=text.split(' ')
    words_clean2 = [stemmer.stem(word) for word in words if word not in stopwords]
    return ' '.join(words_clean2)
docs_clean2 = docs.apply(clean_sentence)
docs_clean2.head()

In [ ]:
docs_clean2.iloc[1]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(10)
vectorizer.fit(docs_clean2)
dtm = vectorizer.transform(docs_clean2)
dtm

In [ ]:
no_of_zeros = (11359*10138 ) - 84319 
sparsity = (no_of_zeros) / (11359*10138 ) * 100
sparsity

In [ ]:
df_dtm = pd.DataFrame(dtm.toarray(),
                     columns = vectorizer.get_feature_names())
(df_dtm == 0).sum().sum()

In [ ]:
 (11359*10138 ) - 84319

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
def get_sentiment_sheldon(text):
    sentiment = SentimentIntensityAnalyzer()
    compound = sentiment.polarity_scores(text)['compound']
    if compound > 0.05:
        return 'Positive'
    elif compound < -0.05:
        return 'Negative'
    else:
        return 'Neutral'
sheldon_dialogues['sentiment_vader'] = sheldon_dialogues['dialogue'].apply(get_sentiment_sheldon)

In [ ]:
sheldon_dialogues['sentiment_vader'].value_counts()

In [ ]:
sheldon_dialogues.columns

In [ ]:
sheldon_dialogues.head()

In [ ]:
df_plot=sheldon_dialogues[['season_id', 'sentiment_vader','dialogue']].groupby(['season_id', 'sentiment_vader']).count()

In [ ]:
df_plot = df_plot.reset_index()

In [ ]:
df_plot.head()

In [ ]:
import seaborn as sns

In [ ]:
sns.lineplot(x='season_id',y='dialogue',hue='sentiment_vader',data=df_plot)
plt.xticks(rotation='vertical')
plt.legend(bbox_to_anchor=(1.5, 1))

In [ ]:
df_plot1=sheldon_dialogues.groupby(['episode_no', 'sentiment_vader']).count()
df_plot1 = df_plot1.reset_index()

In [ ]:
fig = plt.figure(figsize=(20,10))
sns.lineplot(x='episode_no',y='dialogue',hue='sentiment_vader',data=df_plot1)
plt.xticks([])
#plt.locator_params(numticks=15)
plt.legend(bbox_to_anchor=(0.9, 0.9))

In [ ]:
fig = plt.figure(figsize=(10,6))
sns.scatterplot(x='episode_no',y='dialogue',size='dialogue',hue='sentiment_vader',data=df_plot1)
plt.xticks([])
plt.legend(bbox_to_anchor=(1.1, 1))

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
def get_sentiment_agg(text):
    sentiment = SentimentIntensityAnalyzer()
    compound = sentiment.polarity_scores(text)['compound']
    return compound
sheldon_dialogues['numerical_sentiment'] = sheldon_dialogues['dialogue'].apply(get_sentiment_agg)

In [ ]:
df_plot_new=sheldon_dialogues[['season_id','numerical_sentiment']].groupby(['season_id']).sum()
df_plot_new = df_plot_new.reset_index()

In [ ]:
df_plot_new

In [ ]:
sns.lineplot(x='season_id',y='numerical_sentiment',data=df_plot_new)
plt.xticks(rotation='vertical')
#plt.legend(bbox_to_anchor=(1.5, 1))

In [ ]:
df_plot1_new=sheldon_dialogues[['episode_no','numerical_sentiment']].groupby(['episode_no']).sum()
df_plot1_new = df_plot1_new.reset_index()

In [ ]:
k=[]
c=1
for i in range(df_plot1_new.shape[0]):
    c= c+1
    k.append(c)
    
df_plot1_new['video_num'] = k

In [ ]:
fig = plt.figure(figsize=(20,10))
sns.lineplot(x='video_num',y='numerical_sentiment',data=df_plot1_new)